In [ ]:
!pip install requests pandas

In [ ]:
import requests
import pandas as pd
from datetime import datetime

# NOAA API token
token = "CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj"

# NOAA API base URL
base_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/"
headers = {"token": token}

In [ ]:
params = {
    "locationid": "FIPS:36",  # FIPS:36 = New York
    "datasetid": "GHCND",
    "limit": 1000
}

response = requests.get(base_url + "stations", headers=headers, params=params)
stations = response.json()["results"]

# Convert to DataFrame for easier filtering
stations_df = pd.DataFrame(stations)
stations_df.head()


In [ ]:
central_park_station_id = "GHCND:USW00094728"

In [ ]:
import requests
import pandas as pd
from datetime import datetime

In [ ]:
# NOAA API setup
token = "CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj"  # your personal token
headers = {"token": token}
station_id = "GHCND:USW00094728"  # Central Park, NYC
dataset_id = "GHCND"
start_date = "2020-01-01"
end_date = "2020-12-31"


In [ ]:
# Base URL for data
url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"

# Parameters for request
params = {
    "datasetid": dataset_id,
    "stationid": station_id,
    "startdate": start_date,
    "enddate": end_date,
    "limit": 1000,
    "units": "metric",
    "datatypeid": ["TMAX", "TMIN", "PRCP", "SNOW", "AWND"],  # common weather types
    "includemetadata": "false"
}


In [ ]:
# Make initial request
response = requests.get(url, headers=headers, params=params)
data = response.json()["results"]

In [ ]:
# Convert to DataFrame
df = pd.DataFrame(data)


In [ ]:
# Pivot the DataFrame so each datatypeid becomes a column
df_pivot = df.pivot(index="date", columns="datatype", values="value").reset_index()
df_pivot.columns.name = None  # remove metadata name
df_pivot["date"] = pd.to_datetime(df_pivot["date"])



In [ ]:
# Sort by date
df_pivot = df_pivot.sort_values("date")


In [ ]:
# Save to CSV
csv_filename = "weather_central_park_2020.csv"
df_pivot.to_csv(csv_filename, index=False)


In [ ]:
print(f"✅ Saved daily weather data to {csv_filename}")
df_pivot.head()

In [ ]:
import requests

In [ ]:
# Your NOAA Token
NOAA_TOKEN = "CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj"

In [ ]:
# Define parameters for the station search
params = {
    "locationid": "CITY:US360019",  # New York City location ID
    "datasetid": "GHCND",
    "limit": 1000  # Get a large number of stations
}



In [ ]:
# URL for the NCDC API
BASE_URL = "https://www.ncei.noaa.gov/cdo-web/api/v2/stations"


In [ ]:
# Headers with token
headers = {
    "token": NOAA_TOKEN
}


In [ ]:
# Send the request to get the list of stations
response = requests.get(BASE_URL, headers=headers, params=params)



In [ ]:
if response.status_code == 200:
    stations = response.json()["results"]
    station_ids = [station["id"] for station in stations]
    print("Station IDs:", station_ids)
else:
    print(f"Error fetching stations: {response.status_code}")


In [ ]:
import requests
import os
import time
import pandas as pd


In [ ]:
# Function to fetch weather data from NOAA API
def fetch_weather_data(station_id, year, api_key):
    base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    params = {
        'datasetid': 'GHCND',  # Daily data
        'stationid': f'GHCND:{station_id}',  # Station ID format
        'startdate': f'{year}-01-01',  # Start date (January 1st of the year)
        'enddate': f'{year}-12-31',  # End date (December 31st of the year)
        'datatypeid': 'TMAX,TMIN,PRCP',  # Data types (Temperature Max, Temperature Min, Precipitation)
        'limit': 1000  # Limit results per request (adjust as necessary)
    }
    headers = {
        'token': api_key
    }

    try:
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # If data is returned, return the JSON response
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching data for station {station_id} for year {year}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Request error for station {station_id} for year {year}: {e}")
        return None


In [ ]:
# Function to save data into CSV
def save_weather_data(data, station_id, year):
    # Check if data exists
    if not data or 'results' not in data:
        print(f"No data to save for station {station_id} for year {year}")
        return

    # Convert to DataFrame for easier handling
    df = pd.DataFrame(data['results'])

    # If data is not empty, save as CSV
    if not df.empty:
        directory = 'weather_data'
        if not os.path.exists(directory):
            os.makedirs(directory)

        # Save CSV file with station ID and year in the filename
        filename = f"{directory}/weather_{station_id}_{year}.csv"
        df.to_csv(filename, index=False)
        print(f"Saved data for {station_id} for year {year} to {filename}")
    else:
        print(f"No valid records for station {station_id} for year {year}")


In [ ]:
# Main function to loop through stations and years
def download_weather_data(station_ids, start_year, end_year, api_key):
    for station_id in station_ids:
        for year in range(start_year, end_year + 1):
            print(f"Fetching data for station {station_id} for year {year}...")
            data = fetch_weather_data(station_id, year, api_key)

            if data:
                save_weather_data(data, station_id, year)

            # Pause to prevent hitting rate limits (NOAA might have restrictions on number of requests)
            time.sleep(1)


In [ ]:
# List of station IDs related to New York (Replace with your actual station IDs)
station_ids = [
    'GHCND:US1CTFR0022', 'GHCND:US1CTFR0039', 'GHCND:US1NJBG0001', 'GHCND:US1NJBG0002', 'GHCND:US1NJBG0003', 'GHCND:US1NJBG0005', 'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008', 'GHCND:US1NJBG0010', 'GHCND:US1NJBG0011', 'GHCND:US1NJBG0012', 'GHCND:US1NJBG0013', 'GHCND:US1NJBG0015', 'GHCND:US1NJBG0017', 'GHCND:US1NJBG0018', 'GHCND:US1NJBG0020', 'GHCND:US1NJBG0023', 'GHCND:US1NJBG0029', 'GHCND:US1NJBG0030', 'GHCND:US1NJBG0031', 'GHCND:US1NJBG0033', 'GHCND:US1NJBG0037', 'GHCND:US1NJBG0039', 'GHCND:US1NJBG0043', 'GHCND:US1NJBG0044', 'GHCND:US1NJBG0056', 'GHCND:US1NJBG0058', 'GHCND:US1NJBG0060', 'GHCND:US1NJBG0064', 'GHCND:US1NJBG0065', 'GHCND:US1NJBG0070', 'GHCND:US1NJBG0071', 'GHCND:US1NJES0001', 'GHCND:US1NJES0004', 'GHCND:US1NJES0010', 'GHCND:US1NJES0011', 'GHCND:US1NJES0015', 'GHCND:US1NJES0018', 'GHCND:US1NJES0019', 'GHCND:US1NJES0020', 'GHCND:US1NJES0021', 'GHCND:US1NJES0024', 'GHCND:US1NJES0027', 'GHCND:US1NJES0029', 'GHCND:US1NJES0031', 'GHCND:US1NJES0033', 'GHCND:US1NJES0040', 'GHCND:US1NJES0041', 'GHCND:US1NJHD0002', 'GHCND:US1NJHD0016', 'GHCND:US1NJHD0017', 'GHCND:US1NJHD0018', 'GHCND:US1NJHD0023', 'GHCND:US1NJMD0001', 'GHCND:US1NJMD0011', 'GHCND:US1NJMD0023', 'GHCND:US1NJMD0031', 'GHCND:US1NJMD0033', 'GHCND:US1NJMD0038', 'GHCND:US1NJMD0039', 'GHCND:US1NJMD0043', 'GHCND:US1NJMD0044', 'GHCND:US1NJMD0045', 'GHCND:US1NJMD0050', 'GHCND:US1NJMD0055', 'GHCND:US1NJMD0058', 'GHCND:US1NJMD0060', 'GHCND:US1NJMD0062', 'GHCND:US1NJMD0066', 'GHCND:US1NJMD0070', 'GHCND:US1NJMD0071', 'GHCND:US1NJMD0073', 'GHCND:US1NJMD0074', 'GHCND:US1NJMD0082', 'GHCND:US1NJMD0086', 'GHCND:US1NJMD0088', 'GHCND:US1NJMD0093', 'GHCND:US1NJMD0097', 'GHCND:US1NJMD0098', 'GHCND:US1NJMD0104', 'GHCND:US1NJMN0006', 'GHCND:US1NJMN0007', 'GHCND:US1NJMN0010', 'GHCND:US1NJMN0011', 'GHCND:US1NJMN0012', 'GHCND:US1NJMN0015', 'GHCND:US1NJMN0029', 'GHCND:US1NJMN0032', 'GHCND:US1NJMN0033', 'GHCND:US1NJMN0035', 'GHCND:US1NJMN0044', 'GHCND:US1NJMN0048', 'GHCND:US1NJMN0051', 'GHCND:US1NJMN0069', 'GHCND:US1NJMN0081', 'GHCND:US1NJMN0104', 'GHCND:US1NJMS0002', 'GHCND:US1NJMS0011', 'GHCND:US1NJMS0012', 'GHCND:US1NJMS0014', 'GHCND:US1NJMS0016', 'GHCND:US1NJMS0036', 'GHCND:US1NJMS0039', 'GHCND:US1NJMS0040', 'GHCND:US1NJMS0047', 'GHCND:US1NJMS0049', 'GHCND:US1NJMS0058', 'GHCND:US1NJMS0059', 'GHCND:US1NJMS0069', 'GHCND:US1NJMS0070', 'GHCND:US1NJMS0075', 'GHCND:US1NJMS0078', 'GHCND:US1NJMS0089', 'GHCND:US1NJMS0097', 'GHCND:US1NJMS0102', 'GHCND:US1NJMS0106', 'GHCND:US1NJMS0108', 'GHCND:US1NJMS0121', 'GHCND:US1NJPS0003', 'GHCND:US1NJPS0004', 'GHCND:US1NJPS0005', 'GHCND:US1NJPS0008', 'GHCND:US1NJPS0012', 'GHCND:US1NJPS0014', 'GHCND:US1NJPS0015', 'GHCND:US1NJPS0017', 'GHCND:US1NJPS0018', 'GHCND:US1NJPS0019', 'GHCND:US1NJPS0022', 'GHCND:US1NJPS0025', 'GHCND:US1NJPS0028', 'GHCND:US1NJPS0038', 'GHCND:US1NJPS0039', 'GHCND:US1NJPS0040', 'GHCND:US1NJPS0042', 'GHCND:US1NJPS0043', 'GHCND:US1NJSM0099', 'GHCND:US1NJUN0003', 'GHCND:US1NJUN0007', 'GHCND:US1NJUN0010', 'GHCND:US1NJUN0014', 'GHCND:US1NJUN0016', 'GHCND:US1NJUN0017', 'GHCND:US1NJUN0018', 'GHCND:US1NJUN0019', 'GHCND:US1NJUN0020', 'GHCND:US1NJUN0026', 'GHCND:US1NJUN0028', 'GHCND:US1NJUN0034', 'GHCND:US1NJUN0035', 'GHCND:US1NJUN0036', 'GHCND:US1NJUN0038', 'GHCND:US1NJUN0039', 'GHCND:US1NJUN0041', 'GHCND:US1NJUN0042', 'GHCND:US1NYBX0025', 'GHCND:US1NYKN0003', 'GHCND:US1NYKN0025', 'GHCND:US1NYKN0059', 'GHCND:US1NYNS0001', 'GHCND:US1NYNS0006', 'GHCND:US1NYNS0007', 'GHCND:US1NYNS0009', 'GHCND:US1NYNS0014', 'GHCND:US1NYNS0016', 'GHCND:US1NYNS0018', 'GHCND:US1NYNS0022', 'GHCND:US1NYNS0024', 'GHCND:US1NYNS0027', 'GHCND:US1NYNS0028', 'GHCND:US1NYNS0030', 'GHCND:US1NYNS0032', 'GHCND:US1NYNS0034', 'GHCND:US1NYNS0035', 'GHCND:US1NYNS0036', 'GHCND:US1NYNS0037', 'GHCND:US1NYNS0040', 'GHCND:US1NYNS0041', 'GHCND:US1NYNS0042', 'GHCND:US1NYNS0043', 'GHCND:US1NYNS0045', 'GHCND:US1NYNS0046', 'GHCND:US1NYNS0060', 'GHCND:US1NYNS0065', 'GHCND:US1NYNS0066', 'GHCND:US1NYNS0067', 'GHCND:US1NYNS0070', 'GHCND:US1NYNS0071', 'GHCND:US1NYNS0074', 'GHCND:US1NYNS0077', 'GHCND:US1NYNS0080', 'GHCND:US1NYNY0074', 'GHCND:US1NYNY0078', 'GHCND:US1NYNY0082', 'GHCND:US1NYQN0002', 'GHCND:US1NYQN0026', 'GHCND:US1NYQN0027', 'GHCND:US1NYQN0029', 'GHCND:US1NYQN0033', 'GHCND:US1NYQN0036', 'GHCND:US1NYQN0037', 'GHCND:US1NYQN0039', 'GHCND:US1NYRC0001', 'GHCND:US1NYRC0002', 'GHCND:US1NYRC0016', 'GHCND:US1NYRL0001', 'GHCND:US1NYRL0003', 'GHCND:US1NYRL0004', 'GHCND:US1NYRL0005', 'GHCND:US1NYRL0011', 'GHCND:US1NYRL0013', 'GHCND:US1NYSF0041', 'GHCND:US1NYSF0061', 'GHCND:US1NYSF0062', 'GHCND:US1NYSF0075', 'GHCND:US1NYSF0083', 'GHCND:US1NYSF0089', 'GHCND:US1NYSF0092', 'GHCND:US1NYSF0122', 'GHCND:US1NYSF0150', 'GHCND:US1NYSF0158', 'GHCND:US1NYWC0003', 'GHCND:US1NYWC0005', 'GHCND:US1NYWC0009', 'GHCND:US1NYWC0018', 'GHCND:US1NYWC0019', 'GHCND:US1NYWC0032', 'GHCND:US1NYWC0034', 'GHCND:US1NYWC0035', 'GHCND:US1NYWC0049', 'GHCND:USC00063137', 'GHCND:USC00066655', 'GHCND:USC00066954', 'GHCND:USC00280907', 'GHCND:USC00281335', 'GHCND:USC00281472', 'GHCND:USC00281590', 'GHCND:USC00281592', 'GHCND:USC00282023', 'GHCND:USC00282644', 'GHCND:USC00282652', 'GHCND:USC00282768', 'GHCND:USC00282773', 'GHCND:USC00283704', 'GHCND:USC00284260', 'GHCND:USC00284339', 'GHCND:USC00284887', 'GHCND:USC00284931', 'GHCND:USC00284987', 'GHCND:USC00285104', 'GHCND:USC00285503', 'GHCND:USC00286146', 'GHCND:USC00286154', 'GHCND:USC00286560', 'GHCND:USC00286775', 'GHCND:USC00287079', 'GHCND:USC00287393', 'GHCND:USC00287545', 'GHCND:USC00287825', 'GHCND:USC00287833', 'GHCND:USC00287865', 'GHCND:USC00287869', 'GHCND:USC00288423', 'GHCND:USC00288884', 'GHCND:USC00289187', 'GHCND:USC00289271', 'GHCND:USC00289317', 'GHCND:USC00289455', 'GHCND:USC00289832', 'GHCND:USC00300351', 'GHCND:USC00300621', 'GHCND:USC00300958', 'GHCND:USC00300961', 'GHCND:USC00301309', 'GHCND:USC00301618', 'GHCND:USC00302001', 'GHCND:USC00302129', 'GHCND:USC00302291', 'GHCND:USC00302760', 'GHCND:USC00302868', 'GHCND:USC00302964', 'GHCND:USC00303042', 'GHCND:USC00303781', 'GHCND:USC00303786', 'GHCND:USC00303825', 'GHCND:USC00304613', 'GHCND:USC00304632', 'GHCND:USC00305377', 'GHCND:USC00305380', 'GHCND:USC00305382', 'GHCND:USC00305540', 'GHCND:USC00305618', 'GHCND:USC00305796', 'GHCND:USC00305798', 'GHCND:USC00305799', 'GHCND:USC00305804', 'GHCND:USC00305806', 'GHCND:USC00305816', 'GHCND:USC00305821', 'GHCND:USC00306138', 'GHCND:USC00306301', 'GHCND:USC00306368', 'GHCND:USC00306674', 'GHCND:USC00307282', 'GHCND:USC00307338', 'GHCND:USC00307497', 'GHCND:USC00307587', 'GHCND:USC00308072', 'GHCND:USC00308130', 'GHCND:USC00308321', 'GHCND:USC00308322', 'GHCND:USC00308577', 'GHCND:USC00308749', 'GHCND:USC00308946', 'GHCND:USC00309117', 'GHCND:USC00309270', 'GHCND:USC00309400', 'GHCND:USC00309466', 'GHCND:USC00309576', 'GHCND:USC00309580', 'GHCND:USW00014708', 'GHCND:USW00014732', 'GHCND:USW00014734', 'GHCND:USW00014786', 'GHCND:USW00054743', 'GHCND:USW00054787', 'GHCND:USW00094728', 'GHCND:USW00094741', 'GHCND:USW00094745', 'GHCND:USW00094789'
]


In [ ]:
# NOAA API Key (Replace with your actual API key)
api_key = 'CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj'


In [ ]:
# Define the range of years (1995-2024)
start_year = 1995
end_year = 2024

In [ ]:
# Run the download process
download_weather_data(station_ids, start_year, end_year, api_key)


In [ ]:
import requests
import os
import time
import pandas as pd

# Function to fetch weather data from NOAA API
def fetch_weather_data(station_id, year, api_key):
    base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    params = {
        'datasetid': 'GHCND',  # Daily data
        'stationid': f'GHCND:{station_id}',  # Station ID format
        'startdate': f'{year}-01-01',  # Start date (January 1st of the year)
        'enddate': f'{year}-12-31',  # End date (December 31st of the year)
        'datatypeid': 'TMAX,TMIN,PRCP',  # Data types (Temperature Max, Temperature Min, Precipitation)
        'limit': 1000  # Limit results per request (adjust as necessary)
    }
    headers = {
        'token': api_key
    }

    try:
        # Sending the GET request
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Log the status and response for debugging purposes
        print(f"Response status for {station_id} {year}: {response.status_code}")
        
        # If data is returned, return the JSON response
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching data for station {station_id} for year {year}: {response.status_code}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Request error for station {station_id} for year {year}: {e}")
        return None

# Function to save data into CSV
def save_weather_data(data, station_id, year):
    # Check if data exists
    if not data or 'results' not in data or not data['results']:
        print(f"No data available for station {station_id} for year {year}")
        return

    # Convert to DataFrame for easier handling
    df = pd.DataFrame(data['results'])

    # If data is not empty, save as CSV
    if not df.empty:
        directory = 'weather_data'
        if not os.path.exists(directory):
            os.makedirs(directory)

        # Save CSV file with station ID and year in the filename
        filename = f"{directory}/weather_{station_id}_{year}.csv"
        df.to_csv(filename, index=False)
        print(f"Saved data for {station_id} for year {year} to {filename}")
    else:
        print(f"No valid records for station {station_id} for year {year}")

# Main function to loop through stations and years
def download_weather_data(station_ids, start_year, end_year, api_key):
    for station_id in station_ids:
        for year in range(start_year, end_year + 1):
            print(f"Fetching data for station {station_id} for year {year}...")

            # Fetch the weather data
            data = fetch_weather_data(station_id, year, api_key)

            if data:
                # Save the fetched data into CSV
                save_weather_data(data, station_id, year)
            else:
                print(f"No data fetched for station {station_id} for year {year}")

            # Pause to prevent hitting rate limits (NOAA might have restrictions on number of requests)
            time.sleep(1)

# List of station IDs related to New York (Replace with your actual station IDs)
station_ids = [
    'GHCND:US1CTFR0022', 'GHCND:US1CTFR0039', 'GHCND:US1NJBG0001', 'GHCND:US1NJBG0002', 'GHCND:US1NJBG0003', 'GHCND:US1NJBG0005', 'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008', 'GHCND:US1NJBG0010', 'GHCND:US1NJBG0011', 'GHCND:US1NJBG0012', 'GHCND:US1NJBG0013', 'GHCND:US1NJBG0015', 'GHCND:US1NJBG0017', 'GHCND:US1NJBG0018', 'GHCND:US1NJBG0020', 'GHCND:US1NJBG0023', 'GHCND:US1NJBG0029', 'GHCND:US1NJBG0030', 'GHCND:US1NJBG0031', 'GHCND:US1NJBG0033', 'GHCND:US1NJBG0037', 'GHCND:US1NJBG0039', 'GHCND:US1NJBG0043', 'GHCND:US1NJBG0044', 'GHCND:US1NJBG0056', 'GHCND:US1NJBG0058', 'GHCND:US1NJBG0060', 'GHCND:US1NJBG0064', 'GHCND:US1NJBG0065', 'GHCND:US1NJBG0070', 'GHCND:US1NJBG0071', 'GHCND:US1NJES0001', 'GHCND:US1NJES0004', 'GHCND:US1NJES0010', 'GHCND:US1NJES0011', 'GHCND:US1NJES0015', 'GHCND:US1NJES0018', 'GHCND:US1NJES0019', 'GHCND:US1NJES0020', 'GHCND:US1NJES0021', 'GHCND:US1NJES0024', 'GHCND:US1NJES0027', 'GHCND:US1NJES0029', 'GHCND:US1NJES0031', 'GHCND:US1NJES0033', 'GHCND:US1NJES0040', 'GHCND:US1NJES0041', 'GHCND:US1NJHD0002', 'GHCND:US1NJHD0016', 'GHCND:US1NJHD0017', 'GHCND:US1NJHD0018', 'GHCND:US1NJHD0023', 'GHCND:US1NJMD0001', 'GHCND:US1NJMD0011', 'GHCND:US1NJMD0023', 'GHCND:US1NJMD0031', 'GHCND:US1NJMD0033', 'GHCND:US1NJMD0038', 'GHCND:US1NJMD0039', 'GHCND:US1NJMD0043', 'GHCND:US1NJMD0044', 'GHCND:US1NJMD0045', 'GHCND:US1NJMD0050', 'GHCND:US1NJMD0055', 'GHCND:US1NJMD0058', 'GHCND:US1NJMD0060', 'GHCND:US1NJMD0062', 'GHCND:US1NJMD0066', 'GHCND:US1NJMD0070', 'GHCND:US1NJMD0071', 'GHCND:US1NJMD0073', 'GHCND:US1NJMD0074', 'GHCND:US1NJMD0082', 'GHCND:US1NJMD0086', 'GHCND:US1NJMD0088', 'GHCND:US1NJMD0093', 'GHCND:US1NJMD0097', 'GHCND:US1NJMD0098', 'GHCND:US1NJMD0104', 'GHCND:US1NJMN0006', 'GHCND:US1NJMN0007', 'GHCND:US1NJMN0010', 'GHCND:US1NJMN0011', 'GHCND:US1NJMN0012', 'GHCND:US1NJMN0015', 'GHCND:US1NJMN0029', 'GHCND:US1NJMN0032', 'GHCND:US1NJMN0033', 'GHCND:US1NJMN0035', 'GHCND:US1NJMN0044', 'GHCND:US1NJMN0048', 'GHCND:US1NJMN0051', 'GHCND:US1NJMN0069', 'GHCND:US1NJMN0081', 'GHCND:US1NJMN0104', 'GHCND:US1NJMS0002', 'GHCND:US1NJMS0011', 'GHCND:US1NJMS0012', 'GHCND:US1NJMS0014', 'GHCND:US1NJMS0016', 'GHCND:US1NJMS0036', 'GHCND:US1NJMS0039', 'GHCND:US1NJMS0040', 'GHCND:US1NJMS0047', 'GHCND:US1NJMS0049', 'GHCND:US1NJMS0058', 'GHCND:US1NJMS0059', 'GHCND:US1NJMS0069', 'GHCND:US1NJMS0070', 'GHCND:US1NJMS0075', 'GHCND:US1NJMS0078', 'GHCND:US1NJMS0089', 'GHCND:US1NJMS0097', 'GHCND:US1NJMS0102', 'GHCND:US1NJMS0106', 'GHCND:US1NJMS0108', 'GHCND:US1NJMS0121', 'GHCND:US1NJPS0003', 'GHCND:US1NJPS0004', 'GHCND:US1NJPS0005', 'GHCND:US1NJPS0008', 'GHCND:US1NJPS0012', 'GHCND:US1NJPS0014', 'GHCND:US1NJPS0015', 'GHCND:US1NJPS0017', 'GHCND:US1NJPS0018', 'GHCND:US1NJPS0019', 'GHCND:US1NJPS0022', 'GHCND:US1NJPS0025', 'GHCND:US1NJPS0028', 'GHCND:US1NJPS0038', 'GHCND:US1NJPS0039', 'GHCND:US1NJPS0040', 'GHCND:US1NJPS0042', 'GHCND:US1NJPS0043', 'GHCND:US1NJSM0099', 'GHCND:US1NJUN0003', 'GHCND:US1NJUN0007', 'GHCND:US1NJUN0010', 'GHCND:US1NJUN0014', 'GHCND:US1NJUN0016', 'GHCND:US1NJUN0017', 'GHCND:US1NJUN0018', 'GHCND:US1NJUN0019', 'GHCND:US1NJUN0020', 'GHCND:US1NJUN0026', 'GHCND:US1NJUN0028', 'GHCND:US1NJUN0034', 'GHCND:US1NJUN0035', 'GHCND:US1NJUN0036', 'GHCND:US1NJUN0038', 'GHCND:US1NJUN0039', 'GHCND:US1NJUN0041', 'GHCND:US1NJUN0042', 'GHCND:US1NYBX0025', 'GHCND:US1NYKN0003', 'GHCND:US1NYKN0025', 'GHCND:US1NYKN0059', 'GHCND:US1NYNS0001', 'GHCND:US1NYNS0006', 'GHCND:US1NYNS0007', 'GHCND:US1NYNS0009', 'GHCND:US1NYNS0014', 'GHCND:US1NYNS0016', 'GHCND:US1NYNS0018', 'GHCND:US1NYNS0022', 'GHCND:US1NYNS0024', 'GHCND:US1NYNS0027', 'GHCND:US1NYNS0028', 'GHCND:US1NYNS0030', 'GHCND:US1NYNS0032', 'GHCND:US1NYNS0034', 'GHCND:US1NYNS0035', 'GHCND:US1NYNS0036', 'GHCND:US1NYNS0037', 'GHCND:US1NYNS0040', 'GHCND:US1NYNS0041', 'GHCND:US1NYNS0042', 'GHCND:US1NYNS0043', 'GHCND:US1NYNS0045', 'GHCND:US1NYNS0046', 'GHCND:US1NYNS0060', 'GHCND:US1NYNS0065', 'GHCND:US1NYNS0066', 'GHCND:US1NYNS0067', 'GHCND:US1NYNS0070', 'GHCND:US1NYNS0071', 'GHCND:US1NYNS0074', 'GHCND:US1NYNS0077', 'GHCND:US1NYNS0080', 'GHCND:US1NYNY0074', 'GHCND:US1NYNY0078', 'GHCND:US1NYNY0082', 'GHCND:US1NYQN0002', 'GHCND:US1NYQN0026', 'GHCND:US1NYQN0027', 'GHCND:US1NYQN0029', 'GHCND:US1NYQN0033', 'GHCND:US1NYQN0036', 'GHCND:US1NYQN0037', 'GHCND:US1NYQN0039', 'GHCND:US1NYRC0001', 'GHCND:US1NYRC0002', 'GHCND:US1NYRC0016', 'GHCND:US1NYRL0001', 'GHCND:US1NYRL0003', 'GHCND:US1NYRL0004', 'GHCND:US1NYRL0005', 'GHCND:US1NYRL0011', 'GHCND:US1NYRL0013', 'GHCND:US1NYSF0041', 'GHCND:US1NYSF0061', 'GHCND:US1NYSF0062', 'GHCND:US1NYSF0075', 'GHCND:US1NYSF0083', 'GHCND:US1NYSF0089', 'GHCND:US1NYSF0092', 'GHCND:US1NYSF0122', 'GHCND:US1NYSF0150', 'GHCND:US1NYSF0158', 'GHCND:US1NYWC0003', 'GHCND:US1NYWC0005', 'GHCND:US1NYWC0009', 'GHCND:US1NYWC0018', 'GHCND:US1NYWC0019', 'GHCND:US1NYWC0032', 'GHCND:US1NYWC0034', 'GHCND:US1NYWC0035', 'GHCND:US1NYWC0049', 'GHCND:USC00063137', 'GHCND:USC00066655', 'GHCND:USC00066954', 'GHCND:USC00280907', 'GHCND:USC00281335', 'GHCND:USC00281472', 'GHCND:USC00281590', 'GHCND:USC00281592', 'GHCND:USC00282023', 'GHCND:USC00282644', 'GHCND:USC00282652', 'GHCND:USC00282768', 'GHCND:USC00282773', 'GHCND:USC00283704', 'GHCND:USC00284260', 'GHCND:USC00284339', 'GHCND:USC00284887', 'GHCND:USC00284931', 'GHCND:USC00284987', 'GHCND:USC00285104', 'GHCND:USC00285503', 'GHCND:USC00286146', 'GHCND:USC00286154', 'GHCND:USC00286560', 'GHCND:USC00286775', 'GHCND:USC00287079', 'GHCND:USC00287393', 'GHCND:USC00287545', 'GHCND:USC00287825', 'GHCND:USC00287833', 'GHCND:USC00287865', 'GHCND:USC00287869', 'GHCND:USC00288423', 'GHCND:USC00288884', 'GHCND:USC00289187', 'GHCND:USC00289271', 'GHCND:USC00289317', 'GHCND:USC00289455', 'GHCND:USC00289832', 'GHCND:USC00300351', 'GHCND:USC00300621', 'GHCND:USC00300958', 'GHCND:USC00300961', 'GHCND:USC00301309', 'GHCND:USC00301618', 'GHCND:USC00302001', 'GHCND:USC00302129', 'GHCND:USC00302291', 'GHCND:USC00302760', 'GHCND:USC00302868', 'GHCND:USC00302964', 'GHCND:USC00303042', 'GHCND:USC00303781', 'GHCND:USC00303786', 'GHCND:USC00303825', 'GHCND:USC00304613', 'GHCND:USC00304632', 'GHCND:USC00305377', 'GHCND:USC00305380', 'GHCND:USC00305382', 'GHCND:USC00305540', 'GHCND:USC00305618', 'GHCND:USC00305796', 'GHCND:USC00305798', 'GHCND:USC00305799', 'GHCND:USC00305804', 'GHCND:USC00305806', 'GHCND:USC00305816', 'GHCND:USC00305821', 'GHCND:USC00306138', 'GHCND:USC00306301', 'GHCND:USC00306368', 'GHCND:USC00306674', 'GHCND:USC00307282', 'GHCND:USC00307338', 'GHCND:USC00307497', 'GHCND:USC00307587', 'GHCND:USC00308072', 'GHCND:USC00308130', 'GHCND:USC00308321', 'GHCND:USC00308322', 'GHCND:USC00308577', 'GHCND:USC00308749', 'GHCND:USC00308946', 'GHCND:USC00309117', 'GHCND:USC00309270', 'GHCND:USC00309400', 'GHCND:USC00309466', 'GHCND:USC00309576', 'GHCND:USC00309580', 'GHCND:USW00014708', 'GHCND:USW00014732', 'GHCND:USW00014734', 'GHCND:USW00014786', 'GHCND:USW00054743', 'GHCND:USW00054787', 'GHCND:USW00094728', 'GHCND:USW00094741', 'GHCND:USW00094745', 'GHCND:USW00094789'
]

# NOAA API Key (Replace with your actual API key)
api_key = 'CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj'

# Define the range of years (1995-2024)
start_year = 1995
end_year = 2024

# Run the download process
download_weather_data(station_ids, start_year, end_year, api_key)


In [ ]:
import requests
import os
import time
import pandas as pd

# Function to fetch weather data from NOAA API
def fetch_weather_data(station_id, year, api_key):
    base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    params = {
        'datasetid': 'GHCND',  # Daily data
        'stationid': f'GHCND:{station_id}',  # Station ID format
        'startdate': f'{year}-01-01',  # Start date (January 1st of the year)
        'enddate': f'{year}-12-31',  # End date (December 31st of the year)
        'datatypeid': 'TMAX,TMIN,PRCP',  # Data types (Temperature Max, Temperature Min, Precipitation)
        'limit': 1000  # Limit results per request (adjust as necessary)
    }
    headers = {
        'token': api_key
    }

    try:
        # Sending the GET request
        response = requests.get(base_url, params=params, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Log the status and raw response for debugging purposes
        print(f"Response status for {station_id} {year}: {response.status_code}")
        
        # Print the raw response content for debugging
        print("Raw Response Data:")
        print(response.text)

        # If data is returned, return the JSON response
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error fetching data for station {station_id} for year {year}: {response.status_code}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Request error for station {station_id} for year {year}: {e}")
        return None

# Function to save data into CSV
def save_weather_data(data, station_id, year):
    # Check if data exists
    if not data or 'results' not in data or not data['results']:
        print(f"No data available for station {station_id} for year {year}")
        return

    # Convert to DataFrame for easier handling
    df = pd.DataFrame(data['results'])

    # If data is not empty, save as CSV
    if not df.empty:
        directory = 'weather_data'
        if not os.path.exists(directory):
            os.makedirs(directory)

        # Save CSV file with station ID and year in the filename
        filename = f"{directory}/weather_{station_id}_{year}.csv"
        df.to_csv(filename, index=False)
        print(f"Saved data for {station_id} for year {year} to {filename}")
    else:
        print(f"No valid records for station {station_id} for year {year}")

# Main function to loop through stations and years
def download_weather_data(station_ids, start_year, end_year, api_key):
    for station_id in station_ids:
        for year in range(start_year, end_year + 1):
            print(f"Fetching data for station {station_id} for year {year}...")

            # Fetch the weather data
            data = fetch_weather_data(station_id, year, api_key)

            if data:
                # Save the fetched data into CSV
                save_weather_data(data, station_id, year)
            else:
                print(f"No data fetched for station {station_id} for year {year}")

            # Pause to prevent hitting rate limits (NOAA might have restrictions on number of requests)
            time.sleep(1)

# List of station IDs related to New York (Replace with your actual station IDs)
station_ids = [
    'GHCND:US1CTFR0022', 'GHCND:US1CTFR0039', 'GHCND:US1NJBG0001', 'GHCND:US1NJBG0002', 'GHCND:US1NJBG0003', 'GHCND:US1NJBG0005', 'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008', 'GHCND:US1NJBG0010', 'GHCND:US1NJBG0011', 'GHCND:US1NJBG0012', 'GHCND:US1NJBG0013', 'GHCND:US1NJBG0015', 'GHCND:US1NJBG0017', 'GHCND:US1NJBG0018', 'GHCND:US1NJBG0020', 'GHCND:US1NJBG0023', 'GHCND:US1NJBG0029', 'GHCND:US1NJBG0030', 'GHCND:US1NJBG0031', 'GHCND:US1NJBG0033', 'GHCND:US1NJBG0037', 'GHCND:US1NJBG0039', 'GHCND:US1NJBG0043', 'GHCND:US1NJBG0044', 'GHCND:US1NJBG0056', 'GHCND:US1NJBG0058', 'GHCND:US1NJBG0060', 'GHCND:US1NJBG0064', 'GHCND:US1NJBG0065', 'GHCND:US1NJBG0070', 'GHCND:US1NJBG0071', 'GHCND:US1NJES0001', 'GHCND:US1NJES0004', 'GHCND:US1NJES0010', 'GHCND:US1NJES0011', 'GHCND:US1NJES0015', 'GHCND:US1NJES0018', 'GHCND:US1NJES0019', 'GHCND:US1NJES0020', 'GHCND:US1NJES0021', 'GHCND:US1NJES0024', 'GHCND:US1NJES0027', 'GHCND:US1NJES0029', 'GHCND:US1NJES0031', 'GHCND:US1NJES0033', 'GHCND:US1NJES0040', 'GHCND:US1NJES0041', 'GHCND:US1NJHD0002', 'GHCND:US1NJHD0016', 'GHCND:US1NJHD0017', 'GHCND:US1NJHD0018', 'GHCND:US1NJHD0023', 'GHCND:US1NJMD0001', 'GHCND:US1NJMD0011', 'GHCND:US1NJMD0023', 'GHCND:US1NJMD0031', 'GHCND:US1NJMD0033', 'GHCND:US1NJMD0038', 'GHCND:US1NJMD0039', 'GHCND:US1NJMD0043', 'GHCND:US1NJMD0044', 'GHCND:US1NJMD0045', 'GHCND:US1NJMD0050', 'GHCND:US1NJMD0055', 'GHCND:US1NJMD0058', 'GHCND:US1NJMD0060', 'GHCND:US1NJMD0062', 'GHCND:US1NJMD0066', 'GHCND:US1NJMD0070', 'GHCND:US1NJMD0071', 'GHCND:US1NJMD0073', 'GHCND:US1NJMD0074', 'GHCND:US1NJMD0082', 'GHCND:US1NJMD0086', 'GHCND:US1NJMD0088', 'GHCND:US1NJMD0093', 'GHCND:US1NJMD0097', 'GHCND:US1NJMD0098', 'GHCND:US1NJMD0104', 'GHCND:US1NJMN0006', 'GHCND:US1NJMN0007', 'GHCND:US1NJMN0010', 'GHCND:US1NJMN0011', 'GHCND:US1NJMN0012', 'GHCND:US1NJMN0015', 'GHCND:US1NJMN0029', 'GHCND:US1NJMN0032', 'GHCND:US1NJMN0033', 'GHCND:US1NJMN0035', 'GHCND:US1NJMN0044', 'GHCND:US1NJMN0048', 'GHCND:US1NJMN0051', 'GHCND:US1NJMN0069', 'GHCND:US1NJMN0081', 'GHCND:US1NJMN0104', 'GHCND:US1NJMS0002', 'GHCND:US1NJMS0011', 'GHCND:US1NJMS0012', 'GHCND:US1NJMS0014', 'GHCND:US1NJMS0016', 'GHCND:US1NJMS0036', 'GHCND:US1NJMS0039', 'GHCND:US1NJMS0040', 'GHCND:US1NJMS0047', 'GHCND:US1NJMS0049', 'GHCND:US1NJMS0058', 'GHCND:US1NJMS0059', 'GHCND:US1NJMS0069', 'GHCND:US1NJMS0070', 'GHCND:US1NJMS0075', 'GHCND:US1NJMS0078', 'GHCND:US1NJMS0089', 'GHCND:US1NJMS0097', 'GHCND:US1NJMS0102', 'GHCND:US1NJMS0106', 'GHCND:US1NJMS0108', 'GHCND:US1NJMS0121', 'GHCND:US1NJPS0003', 'GHCND:US1NJPS0004', 'GHCND:US1NJPS0005', 'GHCND:US1NJPS0008', 'GHCND:US1NJPS0012', 'GHCND:US1NJPS0014', 'GHCND:US1NJPS0015', 'GHCND:US1NJPS0017', 'GHCND:US1NJPS0018', 'GHCND:US1NJPS0019', 'GHCND:US1NJPS0022', 'GHCND:US1NJPS0025', 'GHCND:US1NJPS0028', 'GHCND:US1NJPS0038', 'GHCND:US1NJPS0039', 'GHCND:US1NJPS0040', 'GHCND:US1NJPS0042', 'GHCND:US1NJPS0043', 'GHCND:US1NJSM0099', 'GHCND:US1NJUN0003', 'GHCND:US1NJUN0007', 'GHCND:US1NJUN0010', 'GHCND:US1NJUN0014', 'GHCND:US1NJUN0016', 'GHCND:US1NJUN0017', 'GHCND:US1NJUN0018', 'GHCND:US1NJUN0019', 'GHCND:US1NJUN0020', 'GHCND:US1NJUN0026', 'GHCND:US1NJUN0028', 'GHCND:US1NJUN0034', 'GHCND:US1NJUN0035', 'GHCND:US1NJUN0036', 'GHCND:US1NJUN0038', 'GHCND:US1NJUN0039', 'GHCND:US1NJUN0041', 'GHCND:US1NJUN0042', 'GHCND:US1NYBX0025', 'GHCND:US1NYKN0003', 'GHCND:US1NYKN0025', 'GHCND:US1NYKN0059', 'GHCND:US1NYNS0001', 'GHCND:US1NYNS0006', 'GHCND:US1NYNS0007', 'GHCND:US1NYNS0009', 'GHCND:US1NYNS0014', 'GHCND:US1NYNS0016', 'GHCND:US1NYNS0018', 'GHCND:US1NYNS0022', 'GHCND:US1NYNS0024', 'GHCND:US1NYNS0027', 'GHCND:US1NYNS0028', 'GHCND:US1NYNS0030', 'GHCND:US1NYNS0032', 'GHCND:US1NYNS0034', 'GHCND:US1NYNS0035', 'GHCND:US1NYNS0036', 'GHCND:US1NYNS0037', 'GHCND:US1NYNS0040', 'GHCND:US1NYNS0041', 'GHCND:US1NYNS0042', 'GHCND:US1NYNS0043', 'GHCND:US1NYNS0045', 'GHCND:US1NYNS0046', 'GHCND:US1NYNS0060', 'GHCND:US1NYNS0065', 'GHCND:US1NYNS0066', 'GHCND:US1NYNS0067', 'GHCND:US1NYNS0070', 'GHCND:US1NYNS0071', 'GHCND:US1NYNS0074', 'GHCND:US1NYNS0077', 'GHCND:US1NYNS0080', 'GHCND:US1NYNY0074', 'GHCND:US1NYNY0078', 'GHCND:US1NYNY0082', 'GHCND:US1NYQN0002', 'GHCND:US1NYQN0026', 'GHCND:US1NYQN0027', 'GHCND:US1NYQN0029', 'GHCND:US1NYQN0033', 'GHCND:US1NYQN0036', 'GHCND:US1NYQN0037', 'GHCND:US1NYQN0039', 'GHCND:US1NYRC0001', 'GHCND:US1NYRC0002', 'GHCND:US1NYRC0016', 'GHCND:US1NYRL0001', 'GHCND:US1NYRL0003', 'GHCND:US1NYRL0004', 'GHCND:US1NYRL0005', 'GHCND:US1NYRL0011', 'GHCND:US1NYRL0013', 'GHCND:US1NYSF0041', 'GHCND:US1NYSF0061', 'GHCND:US1NYSF0062', 'GHCND:US1NYSF0075', 'GHCND:US1NYSF0083', 'GHCND:US1NYSF0089', 'GHCND:US1NYSF0092', 'GHCND:US1NYSF0122', 'GHCND:US1NYSF0150', 'GHCND:US1NYSF0158', 'GHCND:US1NYWC0003', 'GHCND:US1NYWC0005', 'GHCND:US1NYWC0009', 'GHCND:US1NYWC0018', 'GHCND:US1NYWC0019', 'GHCND:US1NYWC0032', 'GHCND:US1NYWC0034', 'GHCND:US1NYWC0035', 'GHCND:US1NYWC0049', 'GHCND:USC00063137', 'GHCND:USC00066655', 'GHCND:USC00066954', 'GHCND:USC00280907', 'GHCND:USC00281335', 'GHCND:USC00281472', 'GHCND:USC00281590', 'GHCND:USC00281592', 'GHCND:USC00282023', 'GHCND:USC00282644', 'GHCND:USC00282652', 'GHCND:USC00282768', 'GHCND:USC00282773', 'GHCND:USC00283704', 'GHCND:USC00284260', 'GHCND:USC00284339', 'GHCND:USC00284887', 'GHCND:USC00284931', 'GHCND:USC00284987', 'GHCND:USC00285104', 'GHCND:USC00285503', 'GHCND:USC00286146', 'GHCND:USC00286154', 'GHCND:USC00286560', 'GHCND:USC00286775', 'GHCND:USC00287079', 'GHCND:USC00287393', 'GHCND:USC00287545', 'GHCND:USC00287825', 'GHCND:USC00287833', 'GHCND:USC00287865', 'GHCND:USC00287869', 'GHCND:USC00288423', 'GHCND:USC00288884', 'GHCND:USC00289187', 'GHCND:USC00289271', 'GHCND:USC00289317', 'GHCND:USC00289455', 'GHCND:USC00289832', 'GHCND:USC00300351', 'GHCND:USC00300621', 'GHCND:USC00300958', 'GHCND:USC00300961', 'GHCND:USC00301309', 'GHCND:USC00301618', 'GHCND:USC00302001', 'GHCND:USC00302129', 'GHCND:USC00302291', 'GHCND:USC00302760', 'GHCND:USC00302868', 'GHCND:USC00302964', 'GHCND:USC00303042', 'GHCND:USC00303781', 'GHCND:USC00303786', 'GHCND:USC00303825', 'GHCND:USC00304613', 'GHCND:USC00304632', 'GHCND:USC00305377', 'GHCND:USC00305380', 'GHCND:USC00305382', 'GHCND:USC00305540', 'GHCND:USC00305618', 'GHCND:USC00305796', 'GHCND:USC00305798', 'GHCND:USC00305799', 'GHCND:USC00305804', 'GHCND:USC00305806', 'GHCND:USC00305816', 'GHCND:USC00305821', 'GHCND:USC00306138', 'GHCND:USC00306301', 'GHCND:USC00306368', 'GHCND:USC00306674', 'GHCND:USC00307282', 'GHCND:USC00307338', 'GHCND:USC00307497', 'GHCND:USC00307587', 'GHCND:USC00308072', 'GHCND:USC00308130', 'GHCND:USC00308321', 'GHCND:USC00308322', 'GHCND:USC00308577', 'GHCND:USC00308749', 'GHCND:USC00308946', 'GHCND:USC00309117', 'GHCND:USC00309270', 'GHCND:USC00309400', 'GHCND:USC00309466', 'GHCND:USC00309576', 'GHCND:USC00309580', 'GHCND:USW00014708', 'GHCND:USW00014732', 'GHCND:USW00014734', 'GHCND:USW00014786', 'GHCND:USW00054743', 'GHCND:USW00054787', 'GHCND:USW00094728', 'GHCND:USW00094741', 'GHCND:USW00094745', 'GHCND:USW00094789'
]

# NOAA API Key (Replace with your actual API key)
api_key = 'CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj'

# Define the range of years (1995-2024)
start_year = 1995
end_year = 2024

# Run the download process
download_weather_data(station_ids, start_year, end_year, api_key)


In [ ]:
import requests
import pandas as pd
import time

# NOAA API token
API_TOKEN = 'CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj'  # Replace with your actual token

# Read station IDs from the text file
with open('ny_station_ids.txt', 'r') as file:
    station_ids = [line.strip() for line in file if line.strip()]

# Define the years for which data is needed
years = range(1995, 1997)  # Example: 1995 and 1996

# NOAA API endpoint
BASE_URL = 'https://www.ncei.noaa.gov/cdo-web/api/v2/data'

# Headers for the API request
headers = {'token': API_TOKEN}

# Parameters for the API request
datasetid = 'GHCND'
datatypeid = ['TMAX', 'TMIN', 'PRCP']  # Maximum temp, Minimum temp, Precipitation
units = 'standard'  # Use 'metric' for Celsius and millimeters

# Loop through each station and year to fetch data
for station_id in station_ids:
    for year in years:
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'
        params = {
            'datasetid': datasetid,
            'datatypeid': datatypeid,
            'stationid': station_id,
            'startdate': start_date,
            'enddate': end_date,
            'units': units,
            'limit': 1000,
            'offset': 1
        }
        all_data = []
        while True:
            response = requests.get(BASE_URL, headers=headers, params=params)
            if response.status_code != 200:
                print(f'Error fetching data for {station_id} {year}: {response.status_code}')
                break
            data = response.json().get('results', [])
            if not data:
                print(f'No data for {station_id} {year}')
                break
            all_data.extend(data)
            if len(data) < params['limit']:
                break
            params['offset'] += params['limit']
            time.sleep(1)  # To respect API rate limits
        if all_data:
            df = pd.DataFrame(all_data)
            filename = f'{station_id.replace(":", "_")}_{year}.csv'
            df.to_csv(filename, index=False)
            print(f'Saved data to {filename}')


In [ ]:
import requests
import pandas as pd
import time

# NOAA API token
API_TOKEN = 'CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj'  # Replace with your actual token

# Read station IDs from the text file
with open('ny_station_ids.txt', 'r') as file:
    station_ids = [line.strip() for line in file if line.strip()]

# Define the years for which data is needed
years = range(1997, 1999)  # Example: 1995 and 1996

# NOAA API endpoint
BASE_URL = 'https://www.ncei.noaa.gov/cdo-web/api/v2/data'

# Headers for the API request
headers = {'token': API_TOKEN}

# Parameters for the API request
datasetid = 'GHCND'
datatypeid = ['TMAX', 'TMIN', 'PRCP']  # Maximum temp, Minimum temp, Precipitation
units = 'standard'  # Use 'metric' for Celsius and millimeters

# Loop through each station and year to fetch data
for station_id in station_ids:
    for year in years:
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'
        params = {
            'datasetid': datasetid,
            'datatypeid': datatypeid,
            'stationid': station_id,
            'startdate': start_date,
            'enddate': end_date,
            'units': units,
            'limit': 1000,
            'offset': 1
        }
        all_data = []
        while True:
            response = requests.get(BASE_URL, headers=headers, params=params)
            if response.status_code != 200:
                print(f'Error fetching data for {station_id} {year}: {response.status_code}')
                break
            data = response.json().get('results', [])
            if not data:
                print(f'No data for {station_id} {year}')
                break
            all_data.extend(data)
            if len(data) < params['limit']:
                break
            params['offset'] += params['limit']
            time.sleep(1)  # To respect API rate limits
        if all_data:
            df = pd.DataFrame(all_data)
            filename = f'{station_id.replace(":", "_")}_{year}.csv'
            df.to_csv(filename, index=False)
            print(f'Saved data to {filename}')


In [ ]:
import requests
import pandas as pd
import time

# NOAA API token
API_TOKEN = 'CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj'  # Replace with your actual token

# Read station IDs from the text file
with open('ny_station_ids.txt', 'r') as file:
    station_ids = [line.strip() for line in file if line.strip()]

# Define the years for which data is needed
years = range(1999, 2001)  # Example: 1995 and 1996

# NOAA API endpoint
BASE_URL = 'https://www.ncei.noaa.gov/cdo-web/api/v2/data'

# Headers for the API request
headers = {'token': API_TOKEN}

# Parameters for the API request
datasetid = 'GHCND'
datatypeid = ['TMAX', 'TMIN', 'PRCP']  # Maximum temp, Minimum temp, Precipitation
units = 'standard'  # Use 'metric' for Celsius and millimeters

# Loop through each station and year to fetch data
for station_id in station_ids:
    for year in years:
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'
        params = {
            'datasetid': datasetid,
            'datatypeid': datatypeid,
            'stationid': station_id,
            'startdate': start_date,
            'enddate': end_date,
            'units': units,
            'limit': 1000,
            'offset': 1
        }
        all_data = []
        while True:
            response = requests.get(BASE_URL, headers=headers, params=params)
            if response.status_code != 200:
                print(f'Error fetching data for {station_id} {year}: {response.status_code}')
                break
            data = response.json().get('results', [])
            if not data:
                print(f'No data for {station_id} {year}')
                break
            all_data.extend(data)
            if len(data) < params['limit']:
                break
            params['offset'] += params['limit']
            time.sleep(1)  # To respect API rate limits
        if all_data:
            df = pd.DataFrame(all_data)
            filename = f'{station_id.replace(":", "_")}_{year}.csv'
            df.to_csv(filename, index=False)
            print(f'Saved data to {filename}')


In [ ]:
import requests
import pandas as pd
import time

# NOAA API token
API_TOKEN = 'CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj'  # Replace with your actual token

# Read station IDs from the text file
with open('ny_station_ids.txt', 'r') as file:
    station_ids = [line.strip() for line in file if line.strip()]

# Define the years for which data is needed
years = range(2001, 2003)  # Example: 1995 and 1996

# NOAA API endpoint
BASE_URL = 'https://www.ncei.noaa.gov/cdo-web/api/v2/data'

# Headers for the API request
headers = {'token': API_TOKEN}

# Parameters for the API request
datasetid = 'GHCND'
datatypeid = ['TMAX', 'TMIN', 'PRCP']  # Maximum temp, Minimum temp, Precipitation
units = 'standard'  # Use 'metric' for Celsius and millimeters

# Loop through each station and year to fetch data
for station_id in station_ids:
    for year in years:
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'
        params = {
            'datasetid': datasetid,
            'datatypeid': datatypeid,
            'stationid': station_id,
            'startdate': start_date,
            'enddate': end_date,
            'units': units,
            'limit': 1000,
            'offset': 1
        }
        all_data = []
        while True:
            response = requests.get(BASE_URL, headers=headers, params=params)
            if response.status_code != 200:
                print(f'Error fetching data for {station_id} {year}: {response.status_code}')
                break
            data = response.json().get('results', [])
            if not data:
                print(f'No data for {station_id} {year}')
                break
            all_data.extend(data)
            if len(data) < params['limit']:
                break
            params['offset'] += params['limit']
            time.sleep(1)  # To respect API rate limits
        if all_data:
            df = pd.DataFrame(all_data)
            filename = f'{station_id.replace(":", "_")}_{year}.csv'
            df.to_csv(filename, index=False)
            print(f'Saved data to {filename}')


In [ ]:
import requests
import pandas as pd
import time

# NOAA API token
API_TOKEN = 'CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj'  # Replace with your actual token

# Read station IDs from the text file
with open('ny_station_ids.txt', 'r') as file:
    station_ids = [line.strip() for line in file if line.strip()]

# Define the years for which data is needed
years = range(2003, 2005)  # Example: 1995 and 1996

# NOAA API endpoint
BASE_URL = 'https://www.ncei.noaa.gov/cdo-web/api/v2/data'

# Headers for the API request
headers = {'token': API_TOKEN}

# Parameters for the API request
datasetid = 'GHCND'
datatypeid = ['TMAX', 'TMIN', 'PRCP']  # Maximum temp, Minimum temp, Precipitation
units = 'standard'  # Use 'metric' for Celsius and millimeters

# Loop through each station and year to fetch data
for station_id in station_ids:
    for year in years:
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'
        params = {
            'datasetid': datasetid,
            'datatypeid': datatypeid,
            'stationid': station_id,
            'startdate': start_date,
            'enddate': end_date,
            'units': units,
            'limit': 1000,
            'offset': 1
        }
        all_data = []
        while True:
            response = requests.get(BASE_URL, headers=headers, params=params)
            if response.status_code != 200:
                print(f'Error fetching data for {station_id} {year}: {response.status_code}')
                break
            data = response.json().get('results', [])
            if not data:
                print(f'No data for {station_id} {year}')
                break
            all_data.extend(data)
            if len(data) < params['limit']:
                break
            params['offset'] += params['limit']
            time.sleep(1)  # To respect API rate limits
        if all_data:
            df = pd.DataFrame(all_data)
            filename = f'{station_id.replace(":", "_")}_{year}.csv'
            df.to_csv(filename, index=False)
            print(f'Saved data to {filename}')


In [ ]:
import requests
import pandas as pd
import time

# NOAA API token
API_TOKEN = 'CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj'  # Replace with your actual token

# Read station IDs from the text file
with open('ny_station_ids.txt', 'r') as file:
    station_ids = [line.strip() for line in file if line.strip()]

# Define the years for which data is needed
years = range(2005, 2007)  # Example: 1995 and 1996

# NOAA API endpoint
BASE_URL = 'https://www.ncei.noaa.gov/cdo-web/api/v2/data'

# Headers for the API request
headers = {'token': API_TOKEN}

# Parameters for the API request
datasetid = 'GHCND'
datatypeid = ['TMAX', 'TMIN', 'PRCP']  # Maximum temp, Minimum temp, Precipitation
units = 'standard'  # Use 'metric' for Celsius and millimeters

# Loop through each station and year to fetch data
for station_id in station_ids:
    for year in years:
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'
        params = {
            'datasetid': datasetid,
            'datatypeid': datatypeid,
            'stationid': station_id,
            'startdate': start_date,
            'enddate': end_date,
            'units': units,
            'limit': 1000,
            'offset': 1
        }
        all_data = []
        while True:
            response = requests.get(BASE_URL, headers=headers, params=params)
            if response.status_code != 200:
                print(f'Error fetching data for {station_id} {year}: {response.status_code}')
                break
            data = response.json().get('results', [])
            if not data:
                print(f'No data for {station_id} {year}')
                break
            all_data.extend(data)
            if len(data) < params['limit']:
                break
            params['offset'] += params['limit']
            time.sleep(1)  # To respect API rate limits
        if all_data:
            df = pd.DataFrame(all_data)
            filename = f'{station_id.replace(":", "_")}_{year}.csv'
            df.to_csv(filename, index=False)
            print(f'Saved data to {filename}')


In [ ]:
import requests
import pandas as pd
import time

# NOAA API token
API_TOKEN = 'CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj'  # Replace with your actual token

# Read station IDs from the text file
with open('ny_station_ids.txt', 'r') as file:
    station_ids = [line.strip() for line in file if line.strip()]

# Define the years for which data is needed
years = range(2007, 2009)  # Example: 1995 and 1996

# NOAA API endpoint
BASE_URL = 'https://www.ncei.noaa.gov/cdo-web/api/v2/data'

# Headers for the API request
headers = {'token': API_TOKEN}

# Parameters for the API request
datasetid = 'GHCND'
datatypeid = ['TMAX', 'TMIN', 'PRCP']  # Maximum temp, Minimum temp, Precipitation
units = 'standard'  # Use 'metric' for Celsius and millimeters

# Loop through each station and year to fetch data
for station_id in station_ids:
    for year in years:
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'
        params = {
            'datasetid': datasetid,
            'datatypeid': datatypeid,
            'stationid': station_id,
            'startdate': start_date,
            'enddate': end_date,
            'units': units,
            'limit': 1000,
            'offset': 1
        }
        all_data = []
        while True:
            response = requests.get(BASE_URL, headers=headers, params=params)
            if response.status_code != 200:
                print(f'Error fetching data for {station_id} {year}: {response.status_code}')
                break
            data = response.json().get('results', [])
            if not data:
                print(f'No data for {station_id} {year}')
                break
            all_data.extend(data)
            if len(data) < params['limit']:
                break
            params['offset'] += params['limit']
            time.sleep(1)  # To respect API rate limits
        if all_data:
            df = pd.DataFrame(all_data)
            filename = f'{station_id.replace(":", "_")}_{year}.csv'
            df.to_csv(filename, index=False)
            print(f'Saved data to {filename}')


In [ ]:
import requests
import pandas as pd
import time

# NOAA API token
API_TOKEN = 'CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj'  # Replace with your actual token

# Read station IDs from the text file
with open('ny_station_ids.txt', 'r') as file:
    station_ids = [line.strip() for line in file if line.strip()]

# Define the years for which data is needed
years = range(2009, 2011)  # Example: 1995 and 1996

# NOAA API endpoint
BASE_URL = 'https://www.ncei.noaa.gov/cdo-web/api/v2/data'

# Headers for the API request
headers = {'token': API_TOKEN}

# Parameters for the API request
datasetid = 'GHCND'
datatypeid = ['TMAX', 'TMIN', 'PRCP']  # Maximum temp, Minimum temp, Precipitation
units = 'standard'  # Use 'metric' for Celsius and millimeters

# Loop through each station and year to fetch data
for station_id in station_ids:
    for year in years:
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'
        params = {
            'datasetid': datasetid,
            'datatypeid': datatypeid,
            'stationid': station_id,
            'startdate': start_date,
            'enddate': end_date,
            'units': units,
            'limit': 1000,
            'offset': 1
        }
        all_data = []
        while True:
            response = requests.get(BASE_URL, headers=headers, params=params)
            if response.status_code != 200:
                print(f'Error fetching data for {station_id} {year}: {response.status_code}')
                break
            data = response.json().get('results', [])
            if not data:
                print(f'No data for {station_id} {year}')
                break
            all_data.extend(data)
            if len(data) < params['limit']:
                break
            params['offset'] += params['limit']
            time.sleep(1)  # To respect API rate limits
        if all_data:
            df = pd.DataFrame(all_data)
            filename = f'{station_id.replace(":", "_")}_{year}.csv'
            df.to_csv(filename, index=False)
            print(f'Saved data to {filename}')


In [ ]:
import requests
import pandas as pd
import time

# NOAA API token
API_TOKEN = 'CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj'  # Replace with your actual token

# Read station IDs from the text file
with open('ny_station_ids.txt', 'r') as file:
    station_ids = [line.strip() for line in file if line.strip()]

# Define the years for which data is needed
years = range(2011, 2013)  # Example: 1995 and 1996

# NOAA API endpoint
BASE_URL = 'https://www.ncei.noaa.gov/cdo-web/api/v2/data'

# Headers for the API request
headers = {'token': API_TOKEN}

# Parameters for the API request
datasetid = 'GHCND'
datatypeid = ['TMAX', 'TMIN', 'PRCP']  # Maximum temp, Minimum temp, Precipitation
units = 'standard'  # Use 'metric' for Celsius and millimeters

# Loop through each station and year to fetch data
for station_id in station_ids:
    for year in years:
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'
        params = {
            'datasetid': datasetid,
            'datatypeid': datatypeid,
            'stationid': station_id,
            'startdate': start_date,
            'enddate': end_date,
            'units': units,
            'limit': 1000,
            'offset': 1
        }
        all_data = []
        while True:
            response = requests.get(BASE_URL, headers=headers, params=params)
            if response.status_code != 200:
                print(f'Error fetching data for {station_id} {year}: {response.status_code}')
                break
            data = response.json().get('results', [])
            if not data:
                print(f'No data for {station_id} {year}')
                break
            all_data.extend(data)
            if len(data) < params['limit']:
                break
            params['offset'] += params['limit']
            time.sleep(1)  # To respect API rate limits
        if all_data:
            df = pd.DataFrame(all_data)
            filename = f'{station_id.replace(":", "_")}_{year}.csv'
            df.to_csv(filename, index=False)
            print(f'Saved data to {filename}')


In [ ]:
import requests
import pandas as pd
import time

# NOAA API token
API_TOKEN = 'CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj'  # Replace with your actual token

# Read station IDs from the text file
with open('ny_station_ids.txt', 'r') as file:
    station_ids = [line.strip() for line in file if line.strip()]

# Define the years for which data is needed
years = range(2013, 2016)  # Example: 1995 and 1996

# NOAA API endpoint
BASE_URL = 'https://www.ncei.noaa.gov/cdo-web/api/v2/data'

# Headers for the API request
headers = {'token': API_TOKEN}

# Parameters for the API request
datasetid = 'GHCND'
datatypeid = ['TMAX', 'TMIN', 'PRCP']  # Maximum temp, Minimum temp, Precipitation
units = 'standard'  # Use 'metric' for Celsius and millimeters

# Loop through each station and year to fetch data
for station_id in station_ids:
    for year in years:
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'
        params = {
            'datasetid': datasetid,
            'datatypeid': datatypeid,
            'stationid': station_id,
            'startdate': start_date,
            'enddate': end_date,
            'units': units,
            'limit': 1000,
            'offset': 1
        }
        all_data = []
        while True:
            response = requests.get(BASE_URL, headers=headers, params=params)
            if response.status_code != 200:
                print(f'Error fetching data for {station_id} {year}: {response.status_code}')
                break
            data = response.json().get('results', [])
            if not data:
                print(f'No data for {station_id} {year}')
                break
            all_data.extend(data)
            if len(data) < params['limit']:
                break
            params['offset'] += params['limit']
            time.sleep(1)  # To respect API rate limits
        if all_data:
            df = pd.DataFrame(all_data)
            filename = f'{station_id.replace(":", "_")}_{year}.csv'
            df.to_csv(filename, index=False)
            print(f'Saved data to {filename}')


Error fetching data for GHCND:US1NYQN0033 2014: 429
Error fetching data for GHCND:US1NYQN0033 2015: 429
Error fetching data for GHCND:US1NYQN0036 2013: 429
Error fetching data for GHCND:US1NYQN0036 2014: 429
Error fetching data for GHCND:US1NYQN0036 2015: 429
Error fetching data for GHCND:US1NYQN0037 2013: 429
Error fetching data for GHCND:US1NYQN0037 2014: 429
Error fetching data for GHCND:US1NYQN0037 2015: 429
Error fetching data for GHCND:US1NYQN0039 2013: 429
Error fetching data for GHCND:US1NYQN0039 2014: 429
Error fetching data for GHCND:US1NYQN0039 2015: 429
Error fetching data for GHCND:US1NYRC0001 2013: 429
Error fetching data for GHCND:US1NYRC0001 2014: 429
Error fetching data for GHCND:US1NYRC0001 2015: 429
Error fetching data for GHCND:US1NYRC0002 2013: 429
Error fetching data for GHCND:US1NYRC0002 2014: 429
Error fetching data for GHCND:US1NYRC0002 2015: 429
Error fetching data for GHCND:US1NYRC0016 2013: 429
Error fetching data for GHCND:US1NYRC0016 2014: 429
Error fetchi

Error fetching data for GHCND:USC00286146 2013: 429
Error fetching data for GHCND:USC00286146 2014: 429
Error fetching data for GHCND:USC00286146 2015: 429
Error fetching data for GHCND:USC00286154 2013: 429
Error fetching data for GHCND:USC00286154 2014: 429
Error fetching data for GHCND:USC00286154 2015: 429
Error fetching data for GHCND:USC00286560 2013: 429
Error fetching data for GHCND:USC00286560 2014: 429
Error fetching data for GHCND:USC00286560 2015: 429
Error fetching data for GHCND:USC00286775 2013: 429
Error fetching data for GHCND:USC00286775 2014: 429
Error fetching data for GHCND:USC00286775 2015: 429
Error fetching data for GHCND:USC00287079 2013: 429
Error fetching data for GHCND:USC00287079 2014: 429
Error fetching data for GHCND:USC00287079 2015: 429
Error fetching data for GHCND:USC00287393 2013: 429
Error fetching data for GHCND:USC00287393 2014: 429
Error fetching data for GHCND:USC00287393 2015: 429
Error fetching data for GHCND:USC00287545 2013: 429
Error fetchi

Error fetching data for GHCND:USC00307282 2015: 429
Error fetching data for GHCND:USC00307338 2013: 429
Error fetching data for GHCND:USC00307338 2014: 429
Error fetching data for GHCND:USC00307338 2015: 429
Error fetching data for GHCND:USC00307497 2013: 429
Error fetching data for GHCND:USC00307497 2014: 429
Error fetching data for GHCND:USC00307497 2015: 429
Error fetching data for GHCND:USC00307587 2013: 429
Error fetching data for GHCND:USC00307587 2014: 429
Error fetching data for GHCND:USC00307587 2015: 429
Error fetching data for GHCND:USC00308072 2013: 429
Error fetching data for GHCND:USC00308072 2014: 429
Error fetching data for GHCND:USC00308072 2015: 429
Error fetching data for GHCND:USC00308130 2013: 429
Error fetching data for GHCND:USC00308130 2014: 429
Error fetching data for GHCND:USC00308130 2015: 429
Error fetching data for GHCND:USC00308321 2013: 429
Error fetching data for GHCND:USC00308321 2014: 429
Error fetching data for GHCND:USC00308321 2015: 429
Error fetchi

In [ ]:
import requests
import pandas as pd
import time

# NOAA API token
API_TOKEN = 'CfRzdxtXoKjMyJZWBApmtTocJgzIJcyj'  # Replace with your actual token

# Read station IDs from the text file
with open('ny_station_ids.txt', 'r') as file:
    station_ids = [line.strip() for line in file if line.strip()]

# Define the years for which data is needed
years = range(2016, 2019)  # Example: 1995 and 1996

# NOAA API endpoint
BASE_URL = 'https://www.ncei.noaa.gov/cdo-web/api/v2/data'

# Headers for the API request
headers = {'token': API_TOKEN}

# Parameters for the API request
datasetid = 'GHCND'
datatypeid = ['TMAX', 'TMIN', 'PRCP']  # Maximum temp, Minimum temp, Precipitation
units = 'standard'  # Use 'metric' for Celsius and millimeters

# Loop through each station and year to fetch data
for station_id in station_ids:
    for year in years:
        start_date = f'{year}-01-01'
        end_date = f'{year}-12-31'
        params = {
            'datasetid': datasetid,
            'datatypeid': datatypeid,
            'stationid': station_id,
            'startdate': start_date,
            'enddate': end_date,
            'units': units,
            'limit': 1000,
            'offset': 1
        }
        all_data = []
        while True:
            response = requests.get(BASE_URL, headers=headers, params=params)
            if response.status_code != 200:
                print(f'Error fetching data for {station_id} {year}: {response.status_code}')
                break
            data = response.json().get('results', [])
            if not data:
                print(f'No data for {station_id} {year}')
                break
            all_data.extend(data)
            if len(data) < params['limit']:
                break
            params['offset'] += params['limit']
            time.sleep(1)  # To respect API rate limits
        if all_data:
            df = pd.DataFrame(all_data)
            filename = f'{station_id.replace(":", "_")}_{year}.csv'
            df.to_csv(filename, index=False)
            print(f'Saved data to {filename}')


Error fetching data for GHCND:US1NYKN0059 2018: 429
Error fetching data for GHCND:US1NYNS0001 2016: 429
Error fetching data for GHCND:US1NYNS0001 2017: 429
Error fetching data for GHCND:US1NYNS0001 2018: 429
Error fetching data for GHCND:US1NYNS0006 2016: 429
Error fetching data for GHCND:US1NYNS0006 2017: 429
Error fetching data for GHCND:US1NYNS0006 2018: 429
Error fetching data for GHCND:US1NYNS0007 2016: 429
Error fetching data for GHCND:US1NYNS0007 2017: 429
Error fetching data for GHCND:US1NYNS0007 2018: 429
Error fetching data for GHCND:US1NYNS0009 2016: 429
Error fetching data for GHCND:US1NYNS0009 2017: 429
Error fetching data for GHCND:US1NYNS0009 2018: 429
Error fetching data for GHCND:US1NYNS0014 2016: 429
Error fetching data for GHCND:US1NYNS0014 2017: 429
Error fetching data for GHCND:US1NYNS0014 2018: 429
Error fetching data for GHCND:US1NYNS0016 2016: 429
Error fetching data for GHCND:US1NYNS0016 2017: 429
Error fetching data for GHCND:US1NYNS0016 2018: 429
Error fetchi

Error fetching data for GHCND:US1NYSF0041 2017: 429
Error fetching data for GHCND:US1NYSF0041 2018: 429
Error fetching data for GHCND:US1NYSF0061 2016: 429
Error fetching data for GHCND:US1NYSF0061 2017: 429
Error fetching data for GHCND:US1NYSF0061 2018: 429
Error fetching data for GHCND:US1NYSF0062 2016: 429
Error fetching data for GHCND:US1NYSF0062 2017: 429
Error fetching data for GHCND:US1NYSF0062 2018: 429
Error fetching data for GHCND:US1NYSF0075 2016: 429
Error fetching data for GHCND:US1NYSF0075 2017: 429
Error fetching data for GHCND:US1NYSF0075 2018: 429
Error fetching data for GHCND:US1NYSF0083 2016: 429
Error fetching data for GHCND:US1NYSF0083 2017: 429
Error fetching data for GHCND:US1NYSF0083 2018: 429
Error fetching data for GHCND:US1NYSF0089 2016: 429
Error fetching data for GHCND:US1NYSF0089 2017: 429
Error fetching data for GHCND:US1NYSF0089 2018: 429
Error fetching data for GHCND:US1NYSF0092 2016: 429
Error fetching data for GHCND:US1NYSF0092 2017: 429
Error fetchi

Error fetching data for GHCND:USC00289187 2016: 429
Error fetching data for GHCND:USC00289187 2017: 429
Error fetching data for GHCND:USC00289187 2018: 429
Error fetching data for GHCND:USC00289271 2016: 429
Error fetching data for GHCND:USC00289271 2017: 429
Error fetching data for GHCND:USC00289271 2018: 429
Error fetching data for GHCND:USC00289317 2016: 429
Error fetching data for GHCND:USC00289317 2017: 429
Error fetching data for GHCND:USC00289317 2018: 429
Error fetching data for GHCND:USC00289455 2016: 429
Error fetching data for GHCND:USC00289455 2017: 429
Error fetching data for GHCND:USC00289455 2018: 429
Error fetching data for GHCND:USC00289832 2016: 429
Error fetching data for GHCND:USC00289832 2017: 429
Error fetching data for GHCND:USC00289832 2018: 429
Error fetching data for GHCND:USC00300351 2016: 429
Error fetching data for GHCND:USC00300351 2017: 429
Error fetching data for GHCND:USC00300351 2018: 429
Error fetching data for GHCND:USC00300621 2016: 429
Error fetchi

Error fetching data for GHCND:USC00309466 2016: 429
Error fetching data for GHCND:USC00309466 2017: 429
Error fetching data for GHCND:USC00309466 2018: 429
Error fetching data for GHCND:USC00309576 2016: 429
Error fetching data for GHCND:USC00309576 2017: 429
Error fetching data for GHCND:USC00309576 2018: 429
Error fetching data for GHCND:USC00309580 2016: 429
Error fetching data for GHCND:USC00309580 2017: 429
Error fetching data for GHCND:USC00309580 2018: 429
Error fetching data for GHCND:USW00014708 2016: 429
Error fetching data for GHCND:USW00014708 2017: 429
Error fetching data for GHCND:USW00014708 2018: 429
Error fetching data for GHCND:USW00014732 2016: 429
Error fetching data for GHCND:USW00014732 2017: 429
Error fetching data for GHCND:USW00014732 2018: 429
Error fetching data for GHCND:USW00014734 2016: 429
Error fetching data for GHCND:USW00014734 2017: 429
Error fetching data for GHCND:USW00014734 2018: 429
Error fetching data for GHCND:USW00014786 2016: 429
Error fetchi

In [ ]:
pip install pandas matplotlib seaborn

In [ ]:
%run process_weather_data.py

In [3]:
%run process_weather_data.py

Combined data saved to D:\env_monitoring_platform\backend\weather_data_files\combined_weather_data.csv
